In [1]:
import pandas as pd

#### read data

In [2]:
nutrient_data = pd.read_csv('food_nutrient.csv')
comps = pd.read_csv('comps.csv')
nutrient_code = pd.read_csv('nutrient.csv')

In [3]:
comps = comps[comps.brand_owner == 'JBS USA, LLC']

In [4]:
nutrient_data.head(3)

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
0,13706913,1105904,203,0.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN
1,13706914,1105904,204,93.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN
2,13706915,1105904,205,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN


In [6]:
nutrient_code.head(3)

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0


#### merge data

In [8]:
comps_nutrient = comps.merge(nutrient_data, how = 'inner', on = 'fdc_id')

In [9]:
comps_nutrient.head(3)

,Unnamed: 0,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,...,id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
0,21947,1127851,"JBS USA, LLC",NaN,NaN,76338473991,GROUND BEEF,NaN,112.0,g,...,13340131,303,2.41,NaN,75.0,NaN,NaN,NaN,NaN,NaN
1,21947,1127851,"JBS USA, LLC",NaN,NaN,76338473991,GROUND BEEF,NaN,112.0,g,...,13340134,601,67.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN
2,21947,1127851,"JBS USA, LLC",NaN,NaN,76338473991,GROUND BEEF,NaN,112.0,g,...,13340127,203,19.64,NaN,70.0,NaN,NaN,NaN,NaN,NaN


In [10]:
nutrient_code = nutrient_code[nutrient_code['nutrient_nbr'].notna()]
nutrient_code['nutrient_nbr'] = nutrient_code['nutrient_nbr'].astype(int)
nutrient_code = nutrient_code.rename(columns={"nutrient_nbr": "nutrient_id"})

In [11]:
master_df = comps_nutrient.merge(nutrient_code, how = 'inner', on = 'nutrient_id')

In [12]:
master_df.head(3)

,Unnamed: 0,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,...,derivation_id,min,max,median,footnote,min_year_acquired,id_y,name,unit_name,rank
0,21947,1127851,"JBS USA, LLC",NaN,NaN,76338473991,GROUND BEEF,NaN,112.0,g,...,75.0,NaN,NaN,NaN,NaN,NaN,1089,"Iron, Fe",MG,5400.0
1,22077,1127981,"JBS USA, LLC",NaN,NaN,45310952205,"BEEF, LESS THAN 2% OF: SALT, BROWN SUGAR, SUGA...",NaN,112.0,g,...,75.0,NaN,NaN,NaN,NaN,NaN,1089,"Iron, Fe",MG,5400.0
2,32235,1138139,"JBS USA, LLC",NaN,NaN,76338966707,100% GRASS FED BEEF,NaN,112.0,g,...,75.0,NaN,NaN,NaN,NaN,NaN,1089,"Iron, Fe",MG,5400.0


In [13]:
master_df.columns

Index(['Unnamed: 0', 'fdc_id', 'brand_owner', 'brand_name', 'subbrand_name',
       'gtin_upc', 'ingredients', 'not_a_significant_source_of',
       'serving_size', 'serving_size_unit', 'household_serving_fulltext',
       'branded_food_category', 'data_source', 'package_weight',
       'modified_date', 'available_date', 'market_country',
       'discontinued_date', 'preparation_state_code', 'trade_channel', 'id_x',
       'nutrient_id', 'amount', 'data_points', 'derivation_id', 'min', 'max',
       'median', 'footnote', 'min_year_acquired', 'id_y', 'name', 'unit_name',
       'rank'],
      dtype='object')

In [14]:
target_columns = ['fdc_id', 'brand_owner', 'serving_size', 
        'branded_food_category', 'nutrient_id', 'name', 'amount']

In [15]:
master_df = master_df[target_columns]

In [16]:
master_df.branded_food_category.value_counts().head(5)

Other Meats                   2119
Sausages, Hotdogs & Brats       99
Frozen Patties and Burgers      60
Poultry, Chicken & Turkey       34
Other Frozen Meats              30
Name: branded_food_category, dtype: int64

Here I chose top food category to dive deep into: 'Other Meats'

In [17]:
master_df = master_df[(master_df.branded_food_category == 'Other Meats')]

In [18]:
master_df.serving_size.value_counts()

112.0    1768
151.0     100
113.0      80
170.0      48
141.0      48
227.0      30
135.0      29
142.0      16
Name: serving_size, dtype: int64

I will focus on serving_size = 112 to make sure all the products are on the same scale

In [19]:
master_df = master_df[(master_df.serving_size == 112.0)]

In [20]:
master_df.name.value_counts().head(5)

Iron, Fe                        143
Protein                         143
Fatty acids, total saturated    143
Total lipid (fat)               143
Energy                          143
Name: name, dtype: int64

In [21]:
master_df = master_df[(master_df.name == 'Iron, Fe')|(master_df.name == 'Cholesterol')|(master_df.name == 'Protein')|
                        (master_df.name == 'Fatty acids, total saturated') | (master_df.name == 'Total lipid (fat)')]

In [22]:
master_df.head(5)

,fdc_id,brand_owner,serving_size,branded_food_category,nutrient_id,name,amount
0,1127851,"JBS USA, LLC",112.0,Other Meats,303,"Iron, Fe",2.41
1,1127981,"JBS USA, LLC",112.0,Other Meats,303,"Iron, Fe",1.61
2,1138139,"JBS USA, LLC",112.0,Other Meats,303,"Iron, Fe",2.41
3,1138887,"JBS USA, LLC",112.0,Other Meats,303,"Iron, Fe",1.61
4,1139727,"JBS USA, LLC",112.0,Other Meats,303,"Iron, Fe",2.41


In [23]:
agg_df = master_df.groupby(['name', 'brand_owner']).agg({'amount':'mean'}).reset_index()

In [24]:
agg_df

,name,brand_owner,amount
0,Cholesterol,"JBS USA, LLC",66.909091
1,"Fatty acids, total saturated","JBS USA, LLC",5.431888
2,"Iron, Fe","JBS USA, LLC",1.713357
3,Protein,"JBS USA, LLC",18.905664
4,Total lipid (fat),"JBS USA, LLC",14.029441


#### Get the industry average

In [25]:
all_comps = pd.read_csv('branded_food.csv')

/var/folders/0g/njjqxd9s5_j0lwlpvmx1n_nc0000gn/T/ipykernel_25642/1090602716.py:1: DtypeWarning: Columns (2,3,4,6,9,12,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  all_comps = pd.read_csv('branded_food.csv')


In [26]:
all_comps = all_comps[all_comps.branded_food_category == 'Other Meats']
all_comps = all_comps[all_comps.serving_size == 112.0]

In [27]:
master_all_df = all_comps.merge(nutrient_data, how = 'inner', on = 'fdc_id')
master_all_df = master_all_df.merge(nutrient_code, how = 'inner', on = 'nutrient_id')

In [28]:
master_all_df = master_all_df[(master_all_df.name == 'Iron, Fe')|(master_all_df.name == 'Cholesterol')|(master_all_df.name == 'Protein')|
                                (master_all_df.name == 'Fatty acids, total saturated') | (master_all_df.name == 'Total lipid (fat)')]

In [29]:
agg_all_df = master_all_df.groupby(['name']).agg({'amount':'median'}).reset_index()

In [30]:
agg_all_df

,name,amount
0,Cholesterol,67.00
1,"Fatty acids, total saturated",5.36
2,"Iron, Fe",1.61
3,Protein,18.75
4,Total lipid (fat),13.39


#### Concatenate results for industry average and JBS

In [31]:
output_df = agg_df.merge(agg_all_df, how = 'inner', on = 'name')

In [32]:
output_df = output_df.rename(columns={'name':'nutrient_name', 'amount_x':'amount', 'amount_y':'industry_avg'})

In [33]:
output_df

,nutrient_name,brand_owner,amount,industry_avg
0,Cholesterol,"JBS USA, LLC",66.909091,67.00
1,"Fatty acids, total saturated","JBS USA, LLC",5.431888,5.36
2,"Iron, Fe","JBS USA, LLC",1.713357,1.61
3,Protein,"JBS USA, LLC",18.905664,18.75
4,Total lipid (fat),"JBS USA, LLC",14.029441,13.39


In [34]:
output_df.to_csv('nutrient_comparison.csv')